In [1]:
import pandas as pd
import os
import gc
import datetime
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LinearRegression,LogisticRegression,SGDRegressor, LinearRegression, Ridge
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold,TimeSeriesSplit
import matplotlib.pyplot as plt
import time
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE,BorderlineSMOTE
from imblearn.combine import SMOTETomek
from sklearn.metrics import mean_absolute_percentage_error,accuracy_score, f1_score, roc_auc_score, log_loss,mean_squared_error
import hyperopt
from hyperopt import hp,fmin,tpe,Trials,STATUS_OK
from timeit import default_timer as timer
import csv
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
data=pd.read_excel("C:/工作/数洋/负荷分析/归档材料/重庆行业维度预测使用数据.xlsx",sheet_name=0)

In [3]:
data.columns

Index(['Unnamed: 0', '大工业用电量', '居民生活用电量', '农业生产用电量', '一般工商业用电量', '大工业用电负荷',
       '居民生活用电负荷', '农业生产用电负荷', '一般工商业用电负荷', '城市', '日期', '白天天气', '最高气温', '白天风力',
       '夜晚天气', '最低气温', '夜晚风力', '是否为节假日', '年份', '月份', '2021-大工业用电量-节假日-基础电量',
       '2021-大工业用电量-非节假日-基础电量', '2022-大工业用电量-节假日-基础电量',
       '2022-大工业用电量-非节假日-基础电量', '2021-居民生活用电量-节假日-基础电量',
       '2021-居民生活用电量-非节假日-基础电量', '2022-居民生活用电量-节假日-基础电量',
       '2022-居民生活用电量-非节假日-基础电量', '2021-农业生产用电量-节假日-基础电量',
       '2021-农业生产用电量-非节假日-基础电量', '2022-农业生产用电量-节假日-基础电量',
       '2022-农业生产用电量-非节假日-基础电量', '2021-一般工商业用电量-节假日-基础电量',
       '2021-一般工商业用电量-非节假日-基础电量', '2022-一般工商业用电量-节假日-基础电量',
       '2022-一般工商业用电量-非节假日-基础电量', '一般工商业用电量-气象电量', '大工业用电量-气象电量',
       '居民生活用电量-气象电量', '农业生产用电量-气象电量', '2021-大工业用电负荷-节假日-基础负荷',
       '2021-大工业用电负荷-非节假日-基础负荷', '2022-大工业用电负荷-节假日-基础负荷',
       '2022-大工业用电负荷-非节假日-基础负荷', '2021-居民生活用电负荷-节假日-基础负荷',
       '2021-居民生活用电负荷-非节假日-基础负荷', '2022-居民生活用电负荷-节假日-基础负荷',
       '2022-居民生活用电负荷-非节假日-基础负荷', '2021-农业生产用电负荷-节假日-基

In [4]:
data.head()

,Unnamed: 0,大工业用电量,居民生活用电量,农业生产用电量,一般工商业用电量,大工业用电负荷,居民生活用电负荷,农业生产用电负荷,一般工商业用电负荷,城市,日期,白天天气,最高气温,白天风力,夜晚天气,最低气温,夜晚风力,是否为节假日,年份,月份,2021-大工业用电量-节假日-基础电量,2021-大工业用电量-非节假日-基础电量,2022-大工业用电量-节假日-基础电量,2022-大工业用电量-非节假日-基础电量,2021-居民生活用电量-节假日-基础电量,2021-居民生活用电量-非节假日-基础电量,2022-居民生活用电量-节假日-基础电量,2022-居民生活用电量-非节假日-基础电量,2021-农业生产用电量-节假日-基础电量,2021-农业生产用电量-非节假日-基础电量,2022-农业生产用电量-节假日-基础电量,2022-农业生产用电量-非节假日-基础电量,2021-一般工商业用电量-节假日-基础电量,2021-一般工商业用电量-非节假日-基础电量,2022-一般工商业用电量-节假日-基础电量,2022-一般工商业用电量-非节假日-基础电量,一般工商业用电量-气象电量,大工业用电量-气象电量,居民生活用电量-气象电量,农业生产用电量-气象电量,2021-大工业用电负荷-节假日-基础负荷,2021-大工业用电负荷-非节假日-基础负荷,2022-大工业用电负荷-节假日-基础负荷,2022-大工业用电负荷-非节假日-基础负荷,2021-居民生活用电负荷-节假日-基础负荷,2021-居民生活用电负荷-非节假日-基础负荷,2022-居民生活用电负荷-节假日-基础负荷,2022-居民生活用电负荷-非节假日-基础负荷,2021-农业生产用电负荷-节假日-基础负荷,2021-农业生产用电负荷-非节假日-基础负荷,2022-农业生产用电负荷-节假日-基础负荷,2022-农业生产用电负荷-非节假日-基础负荷,2021-一般工商业用电负荷-节假日-基础负荷,2021-一般工商业用电负荷-非节假日-基础负荷,2022-一般工商业用电负荷-节假日-基础负荷,2022-一般工商业用电负荷-非节假日-基础负荷,一般工商业用电负荷-气象负荷,大工业用电负荷-气象负荷,居民生活用电负荷-气象负荷,农业生产用电负荷-气象负荷
0,61,2541.9953,5837.3993,201.8991,7585.0532,634.9478,53.5959,3.2436,245.9288,重庆,2021-06-01,多云,32,2,多云,23,2,False,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,969.273793,-274.341122,504.769471,48.525776,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,41.870102,75.396707,8.648615,0.453483
1,62,2481.5065,5770.3049,203.0585,7340.8396,660.1219,52.0959,3.4038,228.6920,重庆,2021-06-02,小雨,31,2,小雨,23,2,False,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,725.060193,-334.829922,437.675071,49.685176,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,24.633302,100.570807,7.148615,0.613683
2,63,2433.3925,5665.4903,189.1833,6636.9617,605.8687,45.0186,2.8591,202.6204,重庆,2021-06-03,小雨,23,2,小雨,20,2,False,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,21.182293,-382.943922,332.860471,35.809976,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,-1.438298,46.317607,0.071315,0.068983
3,64,2770.9390,5347.5331,195.2403,7189.2646,645.2844,46.9820,3.3009,229.3787,重庆,2021-06-04,多云,29,2,晴,19,2,False,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,573.485193,-45.397422,14.903271,41.866976,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,25.320002,85.733307,2.034715,0.510783
4,65,2645.6024,5656.7581,207.4610,7117.1816,561.8248,40.5319,3.1065,222.8982,重庆,2021-06-05,晴,32,2,多云,20,2,True,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,716.289940,9.259535,191.933580,56.817850,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,35.733410,86.280185,-0.749900,0.524500


In [5]:
shift_data = [data['大工业用电负荷-气象负荷'].shift(i) for i in range(1,4)]

In [6]:
shift_data = pd.concat(shift_data, axis=1)

In [7]:
shift_data

,大工业用电负荷-气象负荷,大工业用电负荷-气象负荷,大工业用电负荷-气象负荷
0,NaN,NaN,NaN
1,75.396707,NaN,NaN
2,100.570807,75.396707,NaN
3,46.317607,100.570807,75.396707
4,85.733307,46.317607,100.570807
5,86.280185,85.733307,46.317607
6,84.720885,86.280185,85.733307
7,61.353207,84.720885,86.280185
8,66.209107,61.353207,84.720885
9,-162.239393,66.209107,61.353207


In [8]:
shift_data.columns = ['大工业用电前' + str(i) + '天气象负荷' for i in range(1, 4)]

In [9]:
shift_data

,大工业用电前1天气象负荷,大工业用电前2天气象负荷,大工业用电前3天气象负荷
0,NaN,NaN,NaN
1,75.396707,NaN,NaN
2,100.570807,75.396707,NaN
3,46.317607,100.570807,75.396707
4,85.733307,46.317607,100.570807
5,86.280185,85.733307,46.317607
6,84.720885,86.280185,85.733307
7,61.353207,84.720885,86.280185
8,66.209107,61.353207,84.720885
9,-162.239393,66.209107,61.353207


In [10]:
shift_data.fillna(method='bfill', inplace=True)

In [11]:
shift_data

,大工业用电前1天气象负荷,大工业用电前2天气象负荷,大工业用电前3天气象负荷
0,75.396707,75.396707,75.396707
1,75.396707,75.396707,75.396707
2,100.570807,75.396707,75.396707
3,46.317607,100.570807,75.396707
4,85.733307,46.317607,100.570807
5,86.280185,85.733307,46.317607
6,84.720885,86.280185,85.733307
7,61.353207,84.720885,86.280185
8,66.209107,61.353207,84.720885
9,-162.239393,66.209107,61.353207


In [12]:
data = pd.concat([data, shift_data], axis=1)

In [13]:
# 对类别型变量进行编码
def label_encode(series):
    unique = list(series.unique())
    # unique.sort()
    return series.map(dict(zip(unique, range(series.nunique()))))

In [14]:
#对类别型变量进行处理
cat_cols = ['白天天气','夜晚天气'] 
for col in cat_cols:
    data[col] = label_encode(data[col])

In [15]:
data.head()

,Unnamed: 0,大工业用电量,居民生活用电量,农业生产用电量,一般工商业用电量,大工业用电负荷,居民生活用电负荷,农业生产用电负荷,一般工商业用电负荷,城市,日期,白天天气,最高气温,白天风力,夜晚天气,最低气温,夜晚风力,是否为节假日,年份,月份,2021-大工业用电量-节假日-基础电量,2021-大工业用电量-非节假日-基础电量,2022-大工业用电量-节假日-基础电量,2022-大工业用电量-非节假日-基础电量,2021-居民生活用电量-节假日-基础电量,2021-居民生活用电量-非节假日-基础电量,2022-居民生活用电量-节假日-基础电量,2022-居民生活用电量-非节假日-基础电量,2021-农业生产用电量-节假日-基础电量,2021-农业生产用电量-非节假日-基础电量,2022-农业生产用电量-节假日-基础电量,2022-农业生产用电量-非节假日-基础电量,2021-一般工商业用电量-节假日-基础电量,2021-一般工商业用电量-非节假日-基础电量,2022-一般工商业用电量-节假日-基础电量,2022-一般工商业用电量-非节假日-基础电量,一般工商业用电量-气象电量,大工业用电量-气象电量,居民生活用电量-气象电量,农业生产用电量-气象电量,2021-大工业用电负荷-节假日-基础负荷,2021-大工业用电负荷-非节假日-基础负荷,2022-大工业用电负荷-节假日-基础负荷,2022-大工业用电负荷-非节假日-基础负荷,2021-居民生活用电负荷-节假日-基础负荷,2021-居民生活用电负荷-非节假日-基础负荷,2022-居民生活用电负荷-节假日-基础负荷,2022-居民生活用电负荷-非节假日-基础负荷,2021-农业生产用电负荷-节假日-基础负荷,2021-农业生产用电负荷-非节假日-基础负荷,2022-农业生产用电负荷-节假日-基础负荷,2022-农业生产用电负荷-非节假日-基础负荷,2021-一般工商业用电负荷-节假日-基础负荷,2021-一般工商业用电负荷-非节假日-基础负荷,2022-一般工商业用电负荷-节假日-基础负荷,2022-一般工商业用电负荷-非节假日-基础负荷,一般工商业用电负荷-气象负荷,大工业用电负荷-气象负荷,居民生活用电负荷-气象负荷,农业生产用电负荷-气象负荷,大工业用电前1天气象负荷,大工业用电前2天气象负荷,大工业用电前3天气象负荷
0,61,2541.9953,5837.3993,201.8991,7585.0532,634.9478,53.5959,3.2436,245.9288,重庆,2021-06-01,0,32,2,0,23,2,False,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,969.273793,-274.341122,504.769471,48.525776,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,41.870102,75.396707,8.648615,0.453483,75.396707,75.396707,75.396707
1,62,2481.5065,5770.3049,203.0585,7340.8396,660.1219,52.0959,3.4038,228.6920,重庆,2021-06-02,1,31,2,1,23,2,False,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,725.060193,-334.829922,437.675071,49.685176,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,24.633302,100.570807,7.148615,0.613683,75.396707,75.396707,75.396707
2,63,2433.3925,5665.4903,189.1833,6636.9617,605.8687,45.0186,2.8591,202.6204,重庆,2021-06-03,1,23,2,1,20,2,False,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,21.182293,-382.943922,332.860471,35.809976,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,-1.438298,46.317607,0.071315,0.068983,100.570807,75.396707,75.396707
3,64,2770.9390,5347.5331,195.2403,7189.2646,645.2844,46.9820,3.3009,229.3787,重庆,2021-06-04,0,29,2,2,19,2,False,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,573.485193,-45.397422,14.903271,41.866976,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,25.320002,85.733307,2.034715,0.510783,46.317607,100.570807,75.396707
4,65,2645.6024,5656.7581,207.4610,7117.1816,561.8248,40.5319,3.1065,222.8982,重庆,2021-06-05,2,32,2,0,20,2,True,2021,6,2636.342865,2816.336422,6133.060035,6964.82462,5464.82452,5332.629829,5669.75326,5635.806671,150.64315,153.373324,160.14471,166.571466,6400.89166,6615.779407,6026.391605,6441.302785,716.289940,9.259535,191.933580,56.817850,475.544615,559.551093,471.5749,579.386271,41.2818,44.947285,68.68111,72.086456,2.582,2.790117,3.6792,4.19591,187.16479,204.058698,281.388685,315.711202,35.733410,86.280185,-0.749900,0.524500,85.733307,46.317607,100.570807


In [16]:
col_need=[]
for col in range(1,4):
    col_need.append(f"大工业用电前{col}天气象负荷")

In [17]:
col_need

['大工业用电前1天气象负荷', '大工业用电前2天气象负荷', '大工业用电前3天气象负荷']

In [18]:
#节假日
# data=data.loc[data["是否为节假日"]==True,:]
x_train=data[["白天天气","最高气温","白天风力","夜晚天气","最低气温","夜晚风力"]+col_need].reset_index(drop=True)
y_train=data["大工业用电负荷-气象负荷"].reset_index(drop=True)
x_test=x_train.iloc[:10]

In [19]:
x_train

,白天天气,最高气温,白天风力,夜晚天气,最低气温,夜晚风力,大工业用电前1天气象负荷,大工业用电前2天气象负荷,大工业用电前3天气象负荷
0,0,32,2,0,23,2,75.396707,75.396707,75.396707
1,1,31,2,1,23,2,75.396707,75.396707,75.396707
2,1,23,2,1,20,2,100.570807,75.396707,75.396707
3,0,29,2,2,19,2,46.317607,100.570807,75.396707
4,2,32,2,0,20,2,85.733307,46.317607,100.570807
5,0,34,2,0,21,2,86.280185,85.733307,46.317607
6,3,30,2,0,22,2,84.720885,86.280185,85.733307
7,3,30,2,1,22,2,61.353207,84.720885,86.280185
8,1,23,2,1,21,2,66.209107,61.353207,84.720885
9,0,27,2,0,20,2,-162.239393,66.209107,61.353207


In [20]:
len(x_train)

153

In [21]:
y_train.mean()

163.26389280966055

In [23]:
#xgboost模型参数调优
iteration = 0
MAX_EVALS = 5
count=0
def objective(params):
    global iteration,count#定义全局变量
    iteration += 1 
    start = timer()
    cv_scores = []
    train_x=x_train
    train_y=y_train
    test_x=np.zeros(x_test.shape[0])
    tscv = TimeSeriesSplit(max_train_size=100, n_splits=10)
    for i, (train_index, valid_index) in enumerate(tscv.split(train_x)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index],train_y[valid_index]
        xgb_model = xgb.XGBRegressor(**params)
        xgb_model.fit(trn_x, trn_y, eval_set = [(val_x, val_y)], eval_metric="rmse",
                              early_stopping_rounds = 100)
        val_pred = xgb_model.predict(val_x)
        test_pred = xgb_model.predict(x_test)
        test_x+=test_pred/10
        cv_scores.append(np.sqrt(mean_squared_error(val_y, val_pred)))
    test_pred_df=pd.DataFrame(data=test_x)
    count+=1
    test_pred_df.to_csv(f"C:/工作/数洋/第{count}次预测结果.csv")
    best_score = min(cv_scores)#xgb.cv返回的是列表，lgb.cv返回的是字典
    print("best_score",best_score)
    loss = best_score
    run_time = timer() - start
    of_connection = open("C:/工作/数洋/参数调优结果.csv","a")#将调参结果读取到csv文件，监控调参结果
    writer = csv.writer(of_connection)
    writer.writerow([params,loss,iteration,run_time])
    of_connection.close()
    return {"param":params, "loss":loss, "iteration":iteration,"run_time":run_time,"status":STATUS_OK}

#参数空间
space = {
    "n_estimators":hp.randint("n_estimators",100) + 50,
    "max_depth":hp.randint("max_depth",5) + 5,
    "learning_rate":hp.loguniform("learning_rate",np.log(0.01),np.log(0.5)),
    "gamma":hp.quniform("gamma",0.1,1,0.1),
    "min_child_weight":hp.quniform("min_child_weight",1,10,1),
    "subsample":hp.uniform("subsample",0.9,1.0),
    "colsample_bytree":hp.uniform("colsample_bytree",0.9,1.0),
    "reg_alpha":hp.uniform("reg_alpha",0.0,1.0),
    "reg_lambda":hp.uniform("reg_lambda",0.0,1.0)
}

#优化算法
tpe_algorithm = tpe.suggest

#结果追踪
bayes_trials = Trials()

#自动参数调优
xgb_best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = MAX_EVALS, trials = bayes_trials)

************************************ 1 ************************************                                            
[0]	validation_0-rmse:70.76761                                                                                         
[1]	validation_0-rmse:74.51402                                                                                         
[2]	validation_0-rmse:75.15597                                                                                         
[3]	validation_0-rmse:75.87846                                                                                         
[4]	validation_0-rmse:78.92977                                                                                         
[5]	validation_0-rmse:82.33257                                                                                         
[6]	validation_0-rmse:81.62400                                                                                         
[7]	validation_0-rmse:82.16914          

[34]	validation_0-rmse:133.23847                                                                                       
[35]	validation_0-rmse:133.87096                                                                                       
[36]	validation_0-rmse:134.67917                                                                                       
[37]	validation_0-rmse:135.29396                                                                                       
[38]	validation_0-rmse:135.66015                                                                                       
[39]	validation_0-rmse:135.95460                                                                                       
[40]	validation_0-rmse:136.18064                                                                                       
[41]	validation_0-rmse:137.53342                                                                                       
[42]	validation_0-rmse:137.32909        

[65]	validation_0-rmse:110.37094                                                                                       
[66]	validation_0-rmse:109.94670                                                                                       
[67]	validation_0-rmse:109.12524                                                                                       
[68]	validation_0-rmse:108.79560                                                                                       
[69]	validation_0-rmse:108.94117                                                                                       
[70]	validation_0-rmse:108.37297                                                                                       
[71]	validation_0-rmse:108.68471                                                                                       
[72]	validation_0-rmse:108.89670                                                                                       
[73]	validation_0-rmse:108.27066        

[94]	validation_0-rmse:118.71067                                                                                       
[95]	validation_0-rmse:118.71988                                                                                       
[96]	validation_0-rmse:118.84552                                                                                       
[97]	validation_0-rmse:118.77177                                                                                       
[98]	validation_0-rmse:118.76603                                                                                       
[99]	validation_0-rmse:118.82167                                                                                       
[100]	validation_0-rmse:118.88376                                                                                      
[101]	validation_0-rmse:118.75613                                                                                      
[102]	validation_0-rmse:118.88531       

[16]	validation_0-rmse:106.45729                                                                                       
[17]	validation_0-rmse:106.25537                                                                                       
[18]	validation_0-rmse:106.69155                                                                                       
[19]	validation_0-rmse:106.78225                                                                                       
[20]	validation_0-rmse:108.36338                                                                                       
[21]	validation_0-rmse:108.42857                                                                                       
[22]	validation_0-rmse:108.47370                                                                                       
[23]	validation_0-rmse:108.99311                                                                                       
[24]	validation_0-rmse:110.22438        

...

[67]	validation_0-rmse:86.55212                                                                                        
[68]	validation_0-rmse:86.51295                                                                                        
[69]	validation_0-rmse:86.82300                                                                                        
[70]	validation_0-rmse:86.71346                                                                                        
[71]	validation_0-rmse:86.49739                                                                                        
[72]	validation_0-rmse:86.47976                                                                                        
[73]	validation_0-rmse:86.44775                                                                                        
[74]	validation_0-rmse:86.60462                                                                                        
[75]	validation_0-rmse:87.12345         

[11]	validation_0-rmse:80.69178                                                                                        
[12]	validation_0-rmse:79.84491                                                                                        
[13]	validation_0-rmse:80.92511                                                                                        
[14]	validation_0-rmse:81.72596                                                                                        
[15]	validation_0-rmse:81.08102                                                                                        
[16]	validation_0-rmse:83.26071                                                                                        
[17]	validation_0-rmse:83.05377                                                                                        
[18]	validation_0-rmse:83.86246                                                                                        
[19]	validation_0-rmse:83.23906         

[43]	validation_0-rmse:115.74086                                                                                       
[44]	validation_0-rmse:113.80870                                                                                       
[45]	validation_0-rmse:115.44489                                                                                       
[46]	validation_0-rmse:115.14879                                                                                       
[47]	validation_0-rmse:115.20194                                                                                       
[48]	validation_0-rmse:115.86781                                                                                       
[49]	validation_0-rmse:116.15847                                                                                       
[50]	validation_0-rmse:116.91338                                                                                       
************************************ 4 *

[23]	validation_0-rmse:111.20696                                                                                       
[24]	validation_0-rmse:110.81102                                                                                       
[25]	validation_0-rmse:112.15925                                                                                       
[26]	validation_0-rmse:112.34909                                                                                       
[27]	validation_0-rmse:112.00783                                                                                       
[28]	validation_0-rmse:113.59187                                                                                       
[29]	validation_0-rmse:113.70740                                                                                       
[30]	validation_0-rmse:114.33452                                                                                       
[31]	validation_0-rmse:114.22620        

[3]	validation_0-rmse:285.10522                                                                                        
[4]	validation_0-rmse:259.80318                                                                                        
[5]	validation_0-rmse:240.09168                                                                                        
[6]	validation_0-rmse:220.93039                                                                                        
[7]	validation_0-rmse:204.49447                                                                                        
[8]	validation_0-rmse:187.61009                                                                                        
[9]	validation_0-rmse:173.88866                                                                                        
[10]	validation_0-rmse:161.72696                                                                                       
[11]	validation_0-rmse:152.75658        

In [24]:
# xgb = cv_model(x_train, y_train,"xgb")